# Demo of state swapping
### Functions shown
* `conductor.CalTrialSwapper`
* `conductor.StateBasedSwapper`

Here we have very simplistic model of a virtual lab that has a state and a way to configure that state. The state is multi-valued and depends on multiple variables. 

The method `reconfigure` is considered dangerous. Some variables produce runtime exceptions (i.e. `what=2`). Using the swappers also presents opportunities to mess up arguments. Unhandled errors can corrupt the instruments in the lab and/or put them in a state that is inconsistent with the code's model of the state.

While `Laboratory` is virtual, this also works with any kind of experimental implementation.

### The goal of swapping functions
* Permanently change the state
* Temporarily change the state and go back to the orignal, without having to query and handle the original state explicitly
* Handle all errors so the lab state is always consistent
* Raise those errors to you

In [1]:
import lightlab.equipment.conductor as lc

class MyError(Exception):
    pass

class Laboratory():
    def __init__(self, initialState=None):
        self.stateOfTheLab = initialState
        
    def reconfigure(self, what, how, *args):
        if what == 2:
            raise MyError('Cannot do that')
        self.stateOfTheLab = [what + how, what - how]
        print(*args)
        print(self)
        
    def __str__(self):
        return 'state = ' + str(self.stateOfTheLab)

lab = Laboratory()

## case 1: everything good

In [2]:
calSwap = lc.CalTrialSwapper(lab.reconfigure, 
                             (1, 2, 'Phew, calibration mode'), 
                             (3, 5, 'Ok', 'buddy', 'lets try it'))
for _ in range(5):
    calSwap.setCalibration()  # Does not set calibration state more than once
with calSwap.tempCalibration():  # Does not set calibration state again
    print('Doing stuff with', lab.stateOfTheLab)
with calSwap.tempTrial():  # Changes state to trial
    print('Doing other stuff with', lab.stateOfTheLab)
# Automatically changes back

Phew, calibration mode
state = [3, -1]
Doing stuff with [3, -1]
Ok buddy lets try it
state = [8, -2]
Doing other stuff with [8, -2]
Phew, calibration mode
state = [3, -1]


## case 2: runtime error

In [3]:
calSwap = lc.CalTrialSwapper(lab.reconfigure, 
                             (1, 5, 'Phew, calibration mode'), 
                             (2, 4, 'Ok', 'buddy', 'lets try it'))
try:
    with calSwap.tempTrial():
        print('Futility')
except MyError:
    print('An error was swallowed. State should change back')
print(lab)  # Nevertheless we are back to the trial state

Phew, calibration mode
state = [6, -4]
An error was swallowed. State should change back
state = [6, -4]


## case 3: argument error

In [4]:
calSwap = lc.CalTrialSwapper(lab.reconfigure, 
                             (1, 5, 'Phew, calibration mode'), 
                             10)
calSwap.setCalibration()
try:
    with calSwap.tempTrial():
        print('Futility')
except:
    print('An error was swallowed. State should change back')
print(lab)  # Nevertheless we are back to the calibration state

Phew, calibration mode
state = [6, -4]
You have a problem with the arguments to trial
They are:
 : 10
An error was swallowed. State should change back
state = [6, -4]


## case 3 and beyond: generic version

In [5]:
genericSwap = lc.StateBasedSwapper(print, {'a': 'alex', 'b': 'bhavin', 't': 'tommy'})
for _ in range(10):
    genericSwap.setState('a')
with genericSwap.temporarily('b'):
    with genericSwap.temporarily('t'):
        print('Doing stuff')
# Folds out

alex
bhavin
tommy
Doing stuff
bhavin
alex
